In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import AppLayout
import IPython.display as pyDis

from context import community_module
from community_module.community_detection.agglomerativeCommunityDetection import AgglomerativeCommunityDetection
from community_module.community_detection.explainedCommunitiesDetection import ExplainedCommunitiesDetection
from community_module.visualization.gephiVisualization import GephiVisualization

from sklearn.metrics.pairwise import cosine_similarity

In [9]:
#--------------------------------------------------------------------------------------------------------------------------
#    Custom Belief R Similarity (using DAO csv)
#--------------------------------------------------------------------------------------------------------------------------

In [10]:
from dao.dao_csv import DAO_csv
# similarity
from community_module.similarity.hechtBeliefRSimilarityDAO import HechtBeliefRSimilarityDAO

In [11]:
from community_module.community_detection.agglomerativeCommunityDetectionDistanceMatrix import AgglomerativeCommunityDetectionDistanceMatrix
from community_module.community_detection.explainedCommunitiesDetectionDistanceMatrix import ExplainedCommunitiesDetectionDistanceMatrix

In [12]:
daoHecht = DAO_csv('../../data/HECHT/processed/userBeliefR.csv',',')
similarityMeasure = HechtBeliefRSimilarityDAO(daoHecht)
distanceMatrix = similarityMeasure.matrix_distance()

C:\Users\user\root\WORK\python\2 test\prototype-clustering\community_module\similarity\similarityDAO.py:22: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  self.data = self.dao.pandasData()


In [13]:
percentageDefault = 0.78

In [14]:
# Calculate similarity based on that
community_detection = ExplainedCommunitiesDetectionDistanceMatrix(AgglomerativeCommunityDetectionDistanceMatrix, similarityMeasure.data,distanceMatrix)

In [15]:
n_communities, users_communities = community_detection.search_all_communities(percentage=percentageDefault)
#n_communities, users_communities = community_detection.search_all_communities(percentage=0.35)
users_communities

{25: 5,
 26: 3,
 27: 5,
 28: 0,
 29: 2,
 30: 5,
 31: 3,
 32: 0,
 33: 0,
 34: 0,
 35: 5,
 36: 2,
 37: 0,
 38: 5,
 39: 0,
 40: 0,
 41: 5,
 42: 0,
 44: 2,
 45: 4,
 46: 5,
 47: 1,
 48: 0,
 49: 5,
 51: 0,
 52: 0,
 53: 5,
 54: 2,
 55: 0,
 56: 5,
 57: 5,
 58: 5,
 59: 2,
 60: 3,
 61: 1,
 62: 3,
 63: 2,
 64: 2,
 65: 0,
 66: 4,
 67: 2,
 68: 0,
 69: 5,
 70: 1,
 71: 2,
 72: 1,
 73: 0,
 74: 1,
 75: 0,
 76: 3}

In [16]:
hecht_beliefR_pivot_df2 = similarityMeasure.data.copy()
hecht_beliefR_pivot_df2['community'] = users_communities.values()
hecht_beliefR_pivot_df2.reset_index(inplace=True)
hecht_beliefR_pivot_df2

,index,user,beleifR,community
0,25,PH111172,CRealisticPro,5
1,26,PH112020,EReligousNeg,3
2,27,PH112113,CRealisticPro,5
3,28,PH112469,FRealisticNeg,0
4,29,PH112908,ANatPridePro,2
5,30,PH112935,CRealisticPro,5
6,31,PH113257,EReligousNeg,3
7,32,PH113622,FRealisticNeg,0
8,33,PH114060,FRealisticNeg,0
9,34,PH115272,FRealisticNeg,0


In [17]:
# Explicamos comunidades
users_without_community = []

for c in range(n_communities):
        community_data = community_detection.get_community(c, percentage=percentageDefault)
        
        if len(community_data['members']) > 1:
        
            print('---------------------')
            print('COMMUNITY -', community_data['name'])
            print('\t- N. Members:', len(community_data['members']))
            print('\t- Properties:')

            for k in community_data['properties'].keys():
                print('\t\t-', k, community_data['properties'][k])
        else:
            users_without_community.extend(community_data['members'])
            
            
print('---------------------')
print('N. USERS WITHOUT COMMUNITY -', len(users_without_community))

---------------------
COMMUNITY - 0
	- N. Members: 16
	- Properties:
		- beleifR FRealisticNeg
---------------------
COMMUNITY - 1
	- N. Members: 5
	- Properties:
		- beleifR BReligousPro
---------------------
COMMUNITY - 2
	- N. Members: 9
	- Properties:
		- beleifR ANatPridePro
---------------------
COMMUNITY - 3
	- N. Members: 5
	- Properties:
		- beleifR EReligousNeg
---------------------
COMMUNITY - 4
	- N. Members: 2
	- Properties:
		- beleifR DExtremistNeg
---------------------
COMMUNITY - 5
	- N. Members: 13
	- Properties:
		- beleifR CRealisticPro
---------------------
N. USERS WITHOUT COMMUNITY - 0


In [18]:
#--------------------------------------------------------------------------------------------------------------------------
#    Distance Matrix - Complex similarity (HECHT)
#--------------------------------------------------------------------------------------------------------------------------

In [19]:
# similarity measures
from community_module.similarity.complexSimilarityDAO import ComplexSimilarityDAO
from community_module.similarity.hechtBeliefRSimilarityDAO import HechtBeliefRSimilarityDAO
from community_module.similarity.hechtBeliefJSimilarityDAO import HechtBeliefJSimilarityDAO
from community_module.similarity.hechtDemographicReligiousSimilarityDAO import HechtDemographicReligiousSimilarityDAO
from community_module.similarity.hechtDemographicPoliticsSimilarityDAO import HechtDemographicPoliticsSimilarityDAO


In [20]:
similarityDict = {
    HechtBeliefRSimilarityDAO: 0.8,
    HechtBeliefJSimilarityDAO: 0.6,
    HechtDemographicReligiousSimilarityDAO: 0.2,
    HechtDemographicPoliticsSimilarityDAO: 0.2   
}
similarityDict

{community_module.similarity.hechtBeliefRSimilarityDAO.HechtBeliefRSimilarityDAO: 0.8,
 community_module.similarity.hechtBeliefJSimilarityDAO.HechtBeliefJSimilarityDAO: 0.6,
 community_module.similarity.hechtDemographicReligiousSimilarityDAO.HechtDemographicReligiousSimilarityDAO: 0.2,
 community_module.similarity.hechtDemographicPoliticsSimilarityDAO.HechtDemographicPoliticsSimilarityDAO: 0.2}

In [21]:
daoHecht = DAO_csv('../../data/HECHT/processed/hechtClusteringDAO.csv',';')
similarityMeasure = ComplexSimilarityDAO(daoHecht,similarityDict)
similarityMeasure.data

C:\Users\user\root\WORK\python\2 test\prototype-clustering\community_module\similarity\similarityDAO.py:22: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  self.data = self.dao.pandasData()
C:\Users\user\root\WORK\python\2 test\prototype-clustering\community_module\similarity\similarityDAO.py:22: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  self.data = self.dao.pandasData()
C:\Users\user\root\WORK\python\2 test\prototype-clustering\community_module\similarity\similarityDAO.py:22: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  self.data = self.dao.pandasData()
C:\Users\user\root\WORK\python\2 test\prototype-clustering\community_module\similarity\similarityDAO.py:22: FutureWarning: In a future version of pandas a

,user,beleifR,beliefJ,DemographicPolitics,DemographicReligous
25,PH111172,CRealisticPro,Traitor,L,S
26,PH112020,EReligousNeg,CantJudge,R,S
27,PH112113,CRealisticPro,NotTraitor,R,S
28,PH112469,FRealisticNeg,Traitor,DK,M
29,PH112908,ANatPridePro,NotTraitor,DK,S
30,PH112935,CRealisticPro,CantJudge,L,S
31,PH113257,EReligousNeg,NotTraitor,VR,M
32,PH113622,FRealisticNeg,Traitor,R,S
33,PH114060,FRealisticNeg,CantJudge,DK,S
34,PH115272,FRealisticNeg,CantJudge,C,S


In [22]:
similarityMeasure.data.head(2)

,user,beleifR,beliefJ,DemographicPolitics,DemographicReligous
25,PH111172,CRealisticPro,Traitor,L,S
26,PH112020,EReligousNeg,CantJudge,R,S


In [23]:
# Name returns index value
elemA = similarityMeasure.data.iloc[0].name
elemB = similarityMeasure.data.iloc[1].name

In [24]:
distance = similarityMeasure.distance(elemA,elemB)
distance

0.6222222222222223

In [25]:
# Calculate complex distance matrix
distanceMatrix = similarityMeasure.matrix_distance()
distanceMatrix

array([[0.  , 0.62, 0.39, ..., 0.16, 0.47, 0.43],
       [0.62, 0.  , 0.57, ..., 0.67, 0.29, 0.25],
       [0.39, 0.57, 0.  , ..., 0.43, 0.86, 0.82],
       ...,
       [0.16, 0.72, 0.49, ..., 0.  , 0.51, 0.56],
       [0.47, 0.29, 0.86, ..., 0.54, 0.  , 0.04],
       [0.43, 0.25, 0.82, ..., 0.58, 0.04, 0.  ]])

In [26]:
type(distanceMatrix)

numpy.ndarray

In [27]:
distanceMatrix_df = pd.DataFrame(distanceMatrix)
distanceMatrix_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.00,0.62,0.39,0.58,0.53,0.17,0.84,0.50,0.72,0.64,...,0.37,0.81,0.11,0.46,0.31,0.52,0.89,0.16,0.47,0.43
1,0.62,0.00,0.57,0.29,0.62,0.46,0.22,0.21,0.10,0.07,...,0.46,0.24,0.62,0.64,0.40,0.69,0.27,0.67,0.29,0.25
2,0.39,0.57,0.00,0.86,0.14,0.22,0.46,0.78,0.67,0.64,...,0.31,0.47,0.39,0.07,0.26,0.13,0.50,0.43,0.86,0.82
3,0.58,0.35,0.92,0.00,0.72,0.75,0.49,0.14,0.19,0.31,...,0.55,0.47,0.47,0.87,0.66,0.87,0.36,0.43,0.14,0.18
4,0.53,0.68,0.20,0.72,0.00,0.37,0.54,0.80,0.52,0.63,...,0.17,0.47,0.42,0.16,0.28,0.21,0.36,0.38,0.80,0.84
5,0.17,0.46,0.22,0.75,0.37,0.00,0.68,0.67,0.56,0.47,...,0.20,0.64,0.28,0.29,0.14,0.35,0.72,0.32,0.64,0.59
6,0.84,0.22,0.46,0.41,0.46,0.68,0.00,0.43,0.27,0.29,...,0.62,0.13,0.79,0.47,0.62,0.47,0.10,0.83,0.52,0.47
7,0.50,0.21,0.78,0.08,0.74,0.67,0.43,0.00,0.22,0.19,...,0.58,0.36,0.50,0.76,0.52,0.82,0.39,0.46,0.08,0.13
8,0.72,0.16,0.72,0.19,0.52,0.56,0.35,0.28,0.00,0.11,...,0.36,0.28,0.61,0.68,0.47,0.73,0.17,0.57,0.28,0.32
9,0.64,0.07,0.64,0.28,0.61,0.47,0.29,0.19,0.08,0.00,...,0.44,0.17,0.69,0.62,0.38,0.68,0.25,0.65,0.22,0.27


In [28]:
distanceMatrix_df2 = distanceMatrix_df.round(2)
distanceMatrix_df2

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.00,0.62,0.39,0.58,0.53,0.17,0.84,0.50,0.72,0.64,...,0.37,0.81,0.11,0.46,0.31,0.52,0.89,0.16,0.47,0.43
1,0.62,0.00,0.57,0.29,0.62,0.46,0.22,0.21,0.10,0.07,...,0.46,0.24,0.62,0.64,0.40,0.69,0.27,0.67,0.29,0.25
2,0.39,0.57,0.00,0.86,0.14,0.22,0.46,0.78,0.67,0.64,...,0.31,0.47,0.39,0.07,0.26,0.13,0.50,0.43,0.86,0.82
3,0.58,0.35,0.92,0.00,0.72,0.75,0.49,0.14,0.19,0.31,...,0.55,0.47,0.47,0.87,0.66,0.87,0.36,0.43,0.14,0.18
4,0.53,0.68,0.20,0.72,0.00,0.37,0.54,0.80,0.52,0.63,...,0.17,0.47,0.42,0.16,0.28,0.21,0.36,0.38,0.80,0.84
5,0.17,0.46,0.22,0.75,0.37,0.00,0.68,0.67,0.56,0.47,...,0.20,0.64,0.28,0.29,0.14,0.35,0.72,0.32,0.64,0.59
6,0.84,0.22,0.46,0.41,0.46,0.68,0.00,0.43,0.27,0.29,...,0.62,0.13,0.79,0.47,0.62,0.47,0.10,0.83,0.52,0.47
7,0.50,0.21,0.78,0.08,0.74,0.67,0.43,0.00,0.22,0.19,...,0.58,0.36,0.50,0.76,0.52,0.82,0.39,0.46,0.08,0.13
8,0.72,0.16,0.72,0.19,0.52,0.56,0.35,0.28,0.00,0.11,...,0.36,0.28,0.61,0.68,0.47,0.73,0.17,0.57,0.28,0.32
9,0.64,0.07,0.64,0.28,0.61,0.47,0.29,0.19,0.08,0.00,...,0.44,0.17,0.69,0.62,0.38,0.68,0.25,0.65,0.22,0.27


In [29]:
distanceMatrix2 = distanceMatrix_df2.to_numpy()
distanceMatrix2

array([[0.  , 0.62, 0.39, ..., 0.16, 0.47, 0.43],
       [0.62, 0.  , 0.57, ..., 0.67, 0.29, 0.25],
       [0.39, 0.57, 0.  , ..., 0.43, 0.86, 0.82],
       ...,
       [0.16, 0.72, 0.49, ..., 0.  , 0.51, 0.56],
       [0.47, 0.29, 0.86, ..., 0.54, 0.  , 0.04],
       [0.43, 0.25, 0.82, ..., 0.58, 0.04, 0.  ]])

In [1]:
#--------------------------------------------------------------------------------------------------------------------------
#    Complex similarity (HECHT) - Community Detection
#--------------------------------------------------------------------------------------------------------------------------

In [40]:
from community_module.community_detection.agglomerativeCommunityDetectionDistanceMatrix import AgglomerativeCommunityDetectionDistanceMatrix
from community_module.community_detection.explainedCommunitiesDetectionDistanceMatrix import ExplainedCommunitiesDetectionDistanceMatrix


In [112]:
percentageDefault = 0.78
percentageDefault = 0.5

In [113]:
distanceMatrix

array([[0.  , 0.62, 0.39, ..., 0.16, 0.47, 0.43],
       [0.62, 0.  , 0.57, ..., 0.67, 0.29, 0.25],
       [0.39, 0.57, 0.  , ..., 0.43, 0.86, 0.82],
       ...,
       [0.16, 0.72, 0.49, ..., 0.  , 0.51, 0.56],
       [0.47, 0.29, 0.86, ..., 0.54, 0.  , 0.04],
       [0.43, 0.25, 0.82, ..., 0.58, 0.04, 0.  ]])

In [114]:
# Change index to something better
community_detection_df = similarityMeasure.data.set_index('user')
community_detection_df

,beleifR,beliefJ,DemographicPolitics,DemographicReligous
user,,,,
PH111172,CRealisticPro,Traitor,L,S
PH112020,EReligousNeg,CantJudge,R,S
PH112113,CRealisticPro,NotTraitor,R,S
PH112469,FRealisticNeg,Traitor,DK,M
PH112908,ANatPridePro,NotTraitor,DK,S
PH112935,CRealisticPro,CantJudge,L,S
PH113257,EReligousNeg,NotTraitor,VR,M
PH113622,FRealisticNeg,Traitor,R,S
PH114060,FRealisticNeg,CantJudge,DK,S


In [115]:
# Calculate similarity based on that
community_detection = ExplainedCommunitiesDetectionDistanceMatrix(AgglomerativeCommunityDetectionDistanceMatrix, community_detection_df, distanceMatrix)

In [116]:
n_communities, users_communities = community_detection.search_all_communities(percentage=percentageDefault)
#n_communities, users_communities = community_detection.search_all_communities(percentage=0.35)
users_communities

{'PH111172': 2,
 'PH112020': 0,
 'PH112113': 4,
 'PH112469': 0,
 'PH112908': 4,
 'PH112935': 1,
 'PH113257': 3,
 'PH113622': 0,
 'PH114060': 0,
 'PH115272': 0,
 'PH115643': 2,
 'PH115712': 1,
 'PH121103': 3,
 'PH121881': 2,
 'PH12201': 0,
 'PH123120': 3,
 'PH123152': 4,
 'PH123633': 0,
 'PH124671': 1,
 'PH124901': 3,
 'PH124989': 1,
 'PH125433': 4,
 'PH126033': 0,
 'PH231219': 2,
 'PH231637': 3,
 'PH231979': 0,
 'PH232228': 4,
 'PH23272': 2,
 'PH232819': 3,
 'PH233877': 1,
 'PH235355': 1,
 'PH235809': 4,
 'PH235830': 2,
 'PH236328': 3,
 'PH236659': 4,
 'PH241681': 0,
 'PH243071': 4,
 'PH243072': 1,
 'PH24344': 3,
 'PH243832': 0,
 'PH244236': 1,
 'PH244330': 3,
 'PH24503': 2,
 'PH245669': 4,
 'PH246415': 1,
 'PH246799': 4,
 'PH246943': 3,
 'PH247209': 2,
 'PH248703': 0,
 'PH248869': 0}

In [117]:
n_communities

5

In [124]:
hecht_beliefR_pivot_df2 = community_detection_df.copy()
hecht_beliefR_pivot_df2['community'] = users_communities.values()
hecht_beliefR_pivot_df2.reset_index(inplace=True)
hecht_beliefR_pivot_df2

,user,beleifR,beliefJ,DemographicPolitics,DemographicReligous,community
0,PH111172,CRealisticPro,Traitor,L,S,2
1,PH112020,EReligousNeg,CantJudge,R,S,0
2,PH112113,CRealisticPro,NotTraitor,R,S,4
3,PH112469,FRealisticNeg,Traitor,DK,M,0
4,PH112908,ANatPridePro,NotTraitor,DK,S,4
5,PH112935,CRealisticPro,CantJudge,L,S,1
6,PH113257,EReligousNeg,NotTraitor,VR,M,3
7,PH113622,FRealisticNeg,Traitor,R,S,0
8,PH114060,FRealisticNeg,CantJudge,DK,S,0
9,PH115272,FRealisticNeg,CantJudge,C,S,0


In [125]:
# Explicamos comunidades
users_without_community = []

for c in range(n_communities):
        community_data = community_detection.get_community(c, percentage=percentageDefault)
        
        if len(community_data['members']) > 1:
        
            print('---------------------')
            print('COMMUNITY -', community_data['name'])
            print('\t- N. Members:', len(community_data['members']))
            print('\t- Properties:')

            for k in community_data['properties'].keys():
                print('\t\t-', k, community_data['properties'][k])
        else:
            users_without_community.extend(community_data['members'])
            
            
print('---------------------')
print('N. USERS WITHOUT COMMUNITY -', len(users_without_community))

---------------------
COMMUNITY - 0
	- N. Members: 13
	- Properties:
		- beleifR FRealisticNeg
		- beliefJ Traitor
		- DemographicReligous S
---------------------
COMMUNITY - 1
	- N. Members: 9
	- Properties:
		- beleifR ANatPridePro
		- beliefJ CantJudge
		- DemographicReligous S
---------------------
COMMUNITY - 2
	- N. Members: 8
	- Properties:
		- beleifR CRealisticPro
		- beliefJ Traitor
		- DemographicPolitics DK
		- DemographicReligous S
---------------------
COMMUNITY - 3
	- N. Members: 10
	- Properties:
		- beleifR FRealisticNeg
		- beliefJ NotTraitor
		- DemographicPolitics DK
		- DemographicReligous S
---------------------
COMMUNITY - 4
	- N. Members: 10
	- Properties:
		- beliefJ NotTraitor
		- DemographicReligous S
---------------------
N. USERS WITHOUT COMMUNITY - 0


In [126]:
search_df = hecht_beliefR_pivot_df2[(hecht_beliefR_pivot_df2['community'] == 0)]
search_df

,user,beleifR,beliefJ,DemographicPolitics,DemographicReligous,community
1,PH112020,EReligousNeg,CantJudge,R,S,0
3,PH112469,FRealisticNeg,Traitor,DK,M,0
7,PH113622,FRealisticNeg,Traitor,R,S,0
8,PH114060,FRealisticNeg,CantJudge,DK,S,0
9,PH115272,FRealisticNeg,CantJudge,C,S,0
14,PH12201,FRealisticNeg,CantJudge,C,S,0
17,PH123633,FRealisticNeg,Traitor,DK,S,0
22,PH126033,FRealisticNeg,Traitor,DK,S,0
25,PH231979,FRealisticNeg,Traitor,R,S,0
35,PH241681,EReligousNeg,Traitor,DK,S,0


In [187]:
#--------------------------------------------------------------------------------------------------------------------------
#    Complex similarity (HECHT) - Export JSON (with file)
#--------------------------------------------------------------------------------------------------------------------------

In [188]:
from community_module.community_detection.communityJsonGenerator import CommunityJsonGenerator

In [189]:
distanceMatrix

array([[0.  , 0.62, 0.39, ..., 0.16, 0.47, 0.43],
       [0.62, 0.  , 0.57, ..., 0.67, 0.29, 0.25],
       [0.39, 0.57, 0.  , ..., 0.43, 0.86, 0.82],
       ...,
       [0.16, 0.72, 0.49, ..., 0.  , 0.51, 0.56],
       [0.47, 0.29, 0.86, ..., 0.54, 0.  , 0.04],
       [0.43, 0.25, 0.82, ..., 0.58, 0.04, 0.  ]])

In [190]:
distanceMatrix_df = pd.DataFrame(distanceMatrix)
distanceMatrix_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.00,0.62,0.39,0.58,0.53,0.17,0.84,0.50,0.72,0.64,...,0.37,0.81,0.11,0.46,0.31,0.52,0.89,0.16,0.47,0.43
1,0.62,0.00,0.57,0.29,0.62,0.46,0.22,0.21,0.10,0.07,...,0.46,0.24,0.62,0.64,0.40,0.69,0.27,0.67,0.29,0.25
2,0.39,0.57,0.00,0.86,0.14,0.22,0.46,0.78,0.67,0.64,...,0.31,0.47,0.39,0.07,0.26,0.13,0.50,0.43,0.86,0.82
3,0.58,0.35,0.92,0.00,0.72,0.75,0.49,0.14,0.19,0.31,...,0.55,0.47,0.47,0.87,0.66,0.87,0.36,0.43,0.14,0.18
4,0.53,0.68,0.20,0.72,0.00,0.37,0.54,0.80,0.52,0.63,...,0.17,0.47,0.42,0.16,0.28,0.21,0.36,0.38,0.80,0.84
5,0.17,0.46,0.22,0.75,0.37,0.00,0.68,0.67,0.56,0.47,...,0.20,0.64,0.28,0.29,0.14,0.35,0.72,0.32,0.64,0.59
6,0.84,0.22,0.46,0.41,0.46,0.68,0.00,0.43,0.27,0.29,...,0.62,0.13,0.79,0.47,0.62,0.47,0.10,0.83,0.52,0.47
7,0.50,0.21,0.78,0.08,0.74,0.67,0.43,0.00,0.22,0.19,...,0.58,0.36,0.50,0.76,0.52,0.82,0.39,0.46,0.08,0.13
8,0.72,0.16,0.72,0.19,0.52,0.56,0.35,0.28,0.00,0.11,...,0.36,0.28,0.61,0.68,0.47,0.73,0.17,0.57,0.28,0.32
9,0.64,0.07,0.64,0.28,0.61,0.47,0.29,0.19,0.08,0.00,...,0.44,0.17,0.69,0.62,0.38,0.68,0.25,0.65,0.22,0.27


In [191]:
# Group explicit community properties in one column
json_df = hecht_beliefR_pivot_df2.copy()
json_df['id'] = json_df['user']
json_df['label'] = json_df['user']
json_df = json_df.rename(columns={"community":"group"})
columns = ['DemographicPolitics','DemographicReligous']
columns = ['DemographicPolitics','DemographicReligous','beleifR','beliefJ']
json_df['explicit_community'] = json_df[columns].to_dict(orient='records')
json_df



,user,beleifR,beliefJ,DemographicPolitics,DemographicReligous,group,id,label,explicit_community
0,PH111172,CRealisticPro,Traitor,L,S,2,PH111172,PH111172,"{'DemographicPolitics': 'L', 'DemographicRelig..."
1,PH112020,EReligousNeg,CantJudge,R,S,0,PH112020,PH112020,"{'DemographicPolitics': 'R', 'DemographicRelig..."
2,PH112113,CRealisticPro,NotTraitor,R,S,4,PH112113,PH112113,"{'DemographicPolitics': 'R', 'DemographicRelig..."
3,PH112469,FRealisticNeg,Traitor,DK,M,0,PH112469,PH112469,"{'DemographicPolitics': 'DK', 'DemographicReli..."
4,PH112908,ANatPridePro,NotTraitor,DK,S,4,PH112908,PH112908,"{'DemographicPolitics': 'DK', 'DemographicReli..."
5,PH112935,CRealisticPro,CantJudge,L,S,1,PH112935,PH112935,"{'DemographicPolitics': 'L', 'DemographicRelig..."
6,PH113257,EReligousNeg,NotTraitor,VR,M,3,PH113257,PH113257,"{'DemographicPolitics': 'VR', 'DemographicReli..."
7,PH113622,FRealisticNeg,Traitor,R,S,0,PH113622,PH113622,"{'DemographicPolitics': 'R', 'DemographicRelig..."
8,PH114060,FRealisticNeg,CantJudge,DK,S,0,PH114060,PH114060,"{'DemographicPolitics': 'DK', 'DemographicReli..."
9,PH115272,FRealisticNeg,CantJudge,C,S,0,PH115272,PH115272,"{'DemographicPolitics': 'C', 'DemographicRelig..."


In [205]:
jsonGenerator = CommunityJsonGenerator(json_df,community_detection,n_communities,percentageDefault,distanceMatrix)
jsonGenerator.generateJSON("../jsonVisualization/HECHT.json")

In [193]:
#--------------------------------------------------------------------------------------------------------------------------
#    Complex similarity (HECHT) - Export JSON
#--------------------------------------------------------------------------------------------------------------------------

In [160]:
import json
import math

In [161]:
distanceMatrix

array([[0.  , 0.62, 0.39, ..., 0.16, 0.47, 0.43],
       [0.62, 0.  , 0.57, ..., 0.67, 0.29, 0.25],
       [0.39, 0.57, 0.  , ..., 0.43, 0.86, 0.82],
       ...,
       [0.16, 0.72, 0.49, ..., 0.  , 0.51, 0.56],
       [0.47, 0.29, 0.86, ..., 0.54, 0.  , 0.04],
       [0.43, 0.25, 0.82, ..., 0.58, 0.04, 0.  ]])

In [169]:
distanceMatrix_df = pd.DataFrame(distanceMatrix)
distanceMatrix_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.00,0.62,0.39,0.58,0.53,0.17,0.84,0.50,0.72,0.64,...,0.37,0.81,0.11,0.46,0.31,0.52,0.89,0.16,0.47,0.43
1,0.62,0.00,0.57,0.29,0.62,0.46,0.22,0.21,0.10,0.07,...,0.46,0.24,0.62,0.64,0.40,0.69,0.27,0.67,0.29,0.25
2,0.39,0.57,0.00,0.86,0.14,0.22,0.46,0.78,0.67,0.64,...,0.31,0.47,0.39,0.07,0.26,0.13,0.50,0.43,0.86,0.82
3,0.58,0.35,0.92,0.00,0.72,0.75,0.49,0.14,0.19,0.31,...,0.55,0.47,0.47,0.87,0.66,0.87,0.36,0.43,0.14,0.18
4,0.53,0.68,0.20,0.72,0.00,0.37,0.54,0.80,0.52,0.63,...,0.17,0.47,0.42,0.16,0.28,0.21,0.36,0.38,0.80,0.84
5,0.17,0.46,0.22,0.75,0.37,0.00,0.68,0.67,0.56,0.47,...,0.20,0.64,0.28,0.29,0.14,0.35,0.72,0.32,0.64,0.59
6,0.84,0.22,0.46,0.41,0.46,0.68,0.00,0.43,0.27,0.29,...,0.62,0.13,0.79,0.47,0.62,0.47,0.10,0.83,0.52,0.47
7,0.50,0.21,0.78,0.08,0.74,0.67,0.43,0.00,0.22,0.19,...,0.58,0.36,0.50,0.76,0.52,0.82,0.39,0.46,0.08,0.13
8,0.72,0.16,0.72,0.19,0.52,0.56,0.35,0.28,0.00,0.11,...,0.36,0.28,0.61,0.68,0.47,0.73,0.17,0.57,0.28,0.32
9,0.64,0.07,0.64,0.28,0.61,0.47,0.29,0.19,0.08,0.00,...,0.44,0.17,0.69,0.62,0.38,0.68,0.25,0.65,0.22,0.27


In [170]:
# Group explicit community properties in one column
json_df = hecht_beliefR_pivot_df2.copy()
json_df['id'] = json_df['user']
json_df['label'] = json_df['user']
json_df = json_df.rename(columns={"community":"group"})
columns = ['DemographicPolitics','DemographicReligous']
columns = ['DemographicPolitics','DemographicReligous','beleifR','beliefJ']
json_df['explicit_community'] = json_df[columns].to_dict(orient='records')
json_df



,user,beleifR,beliefJ,DemographicPolitics,DemographicReligous,group,id,label,explicit_community
0,PH111172,CRealisticPro,Traitor,L,S,2,PH111172,PH111172,"{'DemographicPolitics': 'L', 'DemographicRelig..."
1,PH112020,EReligousNeg,CantJudge,R,S,0,PH112020,PH112020,"{'DemographicPolitics': 'R', 'DemographicRelig..."
2,PH112113,CRealisticPro,NotTraitor,R,S,4,PH112113,PH112113,"{'DemographicPolitics': 'R', 'DemographicRelig..."
3,PH112469,FRealisticNeg,Traitor,DK,M,0,PH112469,PH112469,"{'DemographicPolitics': 'DK', 'DemographicReli..."
4,PH112908,ANatPridePro,NotTraitor,DK,S,4,PH112908,PH112908,"{'DemographicPolitics': 'DK', 'DemographicReli..."
5,PH112935,CRealisticPro,CantJudge,L,S,1,PH112935,PH112935,"{'DemographicPolitics': 'L', 'DemographicRelig..."
6,PH113257,EReligousNeg,NotTraitor,VR,M,3,PH113257,PH113257,"{'DemographicPolitics': 'VR', 'DemographicReli..."
7,PH113622,FRealisticNeg,Traitor,R,S,0,PH113622,PH113622,"{'DemographicPolitics': 'R', 'DemographicRelig..."
8,PH114060,FRealisticNeg,CantJudge,DK,S,0,PH114060,PH114060,"{'DemographicPolitics': 'DK', 'DemographicReli..."
9,PH115272,FRealisticNeg,CantJudge,C,S,0,PH115272,PH115272,"{'DemographicPolitics': 'C', 'DemographicRelig..."


In [171]:
# Export community information to JSON format
communityJson = {}

In [172]:
# Community Data
communityJson['communities'] = []

for c in range(n_communities):
    community_data = community_detection.get_community(c, percentage=percentageDefault)

    communityDictionary = {}
    communityDictionary['community-type'] = 'implicit'
    communityDictionary['name'] = 'Community ' + str(c)
    
    if len(community_data['members']) > 1:
        communityPropertiesList = []
        for k in community_data['properties'].keys():
            #print('\t\t-', k)
            #communityProperties += '\t\t-' + ' ' + str(k) + ' ' + community_data['properties'][k] + '\n'
            communityPropertiesList.append("'" + str(k) + "'"  + ': ' + "'" + community_data['properties'][k] + "'")
        #communityProperties = 'Similar dominant emotions while interacting with the following artworks: {'
        communityProperties = 'Properties: {'
        communityProperties += '; '.join(communityPropertiesList)
        communityProperties += '}'
        
    else:
        communityProperties = 'Users without community'
        
    communityDictionary['explanation'] = communityProperties
    #communityDictionary[name]['users'] = 
    
    
    communityDictionary['users'] = []
    for user in community_data['members']:
        communityDictionary['users'].append(str(user))
    
    #print(communityDictionary)
    communityJson['communities'].append(communityDictionary)

In [173]:
# User Data
communityJson["users"] = []
communityJson['users'] = json_df[['id','label','group','explicit_community']].to_dict('records')
communityJson

{'communities': [{'community-type': 'implicit',
   'name': 'Community 0',
   'explanation': "Properties: {'beleifR': 'FRealisticNeg'; 'beliefJ': 'Traitor'; 'DemographicReligous': 'S'}",
   'users': ['PH112020',
    'PH112469',
    'PH113622',
    'PH114060',
    'PH115272',
    'PH12201',
    'PH123633',
    'PH126033',
    'PH231979',
    'PH241681',
    'PH243832',
    'PH248703',
    'PH248869']},
  {'community-type': 'implicit',
   'name': 'Community 1',
   'explanation': "Properties: {'beleifR': 'ANatPridePro'; 'beliefJ': 'CantJudge'; 'DemographicReligous': 'S'}",
   'users': ['PH112935',
    'PH115712',
    'PH124671',
    'PH124989',
    'PH233877',
    'PH235355',
    'PH243072',
    'PH244236',
    'PH246415']},
  {'community-type': 'implicit',
   'name': 'Community 2',
   'explanation': "Properties: {'beleifR': 'CRealisticPro'; 'beliefJ': 'Traitor'; 'DemographicPolitics': 'DK'; 'DemographicReligous': 'S'}",
   'users': ['PH111172',
    'PH115643',
    'PH121881',
    'PH23121

In [174]:
# Similarity Data
communityJson['similarity'] = []    
# users
for i in range(len(distanceMatrix)):
    for j in range(i+1,len(distanceMatrix[i])):
        dicti = {}
        dicti['u1'] = str(json_df.iloc[i]['label'])
        dicti['u2'] = str(json_df.iloc[j]['label'])
        #dicti['value'] = similarityMatrix[i][j]
        dicti['value'] = round(1 - distanceMatrix[i][j],2)
        communityJson['similarity'].append(dicti)   

In [175]:
with open("../jsonVisualization/HECHT.json", "w") as outfile:
    json.dump(communityJson, outfile, indent=4)

In [38]:
d = dict([
    (<key>, <value>),
    (<key>, <value),
      .
      .
      .
    (<key>, <value>)
])


SyntaxError: invalid syntax (1054640310.py, line 2)